In [52]:
import pickle
import os
import pytesseract
from PIL import Image
from pdf2image import convert_from_path

In [53]:
pdf_images = convert_from_path("Input/set-1/SP_MIS02824100914340 1.pdf")
image = pdf_images[0].convert("RGB")

In [54]:

# Manually set the full path to tesseract.exe
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"


In [58]:
ocr_data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)

In [59]:
type(ocr_data)
len(ocr_data)

12

In [ ]:
# Extract words and their bounding boxes

words = []
boxes = []

for i in range(len(ocr_data['text'])):
    if ocr_data['text'][i].strip():
        words.append(ocr_data['text'][i])
        x, y, w, h = (ocr_data['left'][i], ocr_data['top'][i], ocr_data['width'][i], ocr_data['height'][i])
        boxes.append([x, y, x+w, y+h])


print('Extracted Words: ', words)
print('Bounding Boxes: ', boxes)

In [49]:
len(words)
len(boxes)

417

In [40]:
# Get image dimensions (width, height)
image_width, image_height = image.size  # Assuming `image` is a PIL Image

# Normalize bounding boxes to fit in [0, 1000] range
normalized_boxes = [
    [
        int((x / image_width) * 1000),   # Normalize x1
        int((y / image_height) * 1000),  # Normalize y1
        int((x_w / image_width) * 1000), # Normalize x2
        int((y_h / image_height) * 1000) # Normalize y2
    ]
    for (x, y, x_w, y_h) in boxes
]

# Debugging: Print some normalized values
print("Original Bounding Boxes:", boxes[:5])
print("Normalized Bounding Boxes:", normalized_boxes[:5])

Original Bounding Boxes: [[896, 90, 1012, 112], [1023, 91, 1144, 113], [1156, 92, 1269, 113], [1278, 104, 1285, 106], [1296, 92, 1350, 113]]
Normalized Bounding Boxes: [[527, 40, 595, 50], [601, 41, 672, 51], [680, 41, 746, 51], [751, 47, 755, 48], [762, 41, 794, 51]]


In [29]:
from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification
import torch

In [ ]:
# Load pre-trained LayoutLMv3 processor and model
processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)
model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base", num_labels=4)

# Tokenization
encoding = processor(images=image, text=words, boxes=normalized_boxes, return_tensors="pt", padding="max_length", truncation=True)

# Run model inference
with torch.no_grad():
    outputs = model(**encoding)

# Get predicted labels
predicted_labels = torch.argmax(outputs.logits, dim=1)
print(predicted_labels)

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\vuvee\anaconda3\envs\dev\Lib\site-packages\transformers\modeling_utils.py:1072: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tensor([[ 16, 269, 474, 506]])
